In [8]:
%pip install -i https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple google-generativeai

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


In [1]:
# --- 安全地设置API密钥 ---
import os
import sys

# 对于本地Jupyter用户：
# 你必须在启动Jupyter Notebook的终端中先设置环境变量。
#
# 在 macOS / Linux 终端中:
# export GOOGLE_API_KEY="你的新密钥粘贴在这里"
#
# 在 Windows 命令提示符 (CMD) 中:
# set GOOGLE_API_KEY="你的新密钥粘贴在这里"
#
# 在 Windows PowerShell 中:
# $env:GOOGLE_API_KEY="你的新密钥粘贴在这里"
#
# 设置完毕后，从【同一个】终端窗口启动 jupyter notebook 或 jupyter lab。

# 这段代码会从环境变量中加载密钥

import google.generativeai as genai

if 'GOOGLE_API_KEY' in os.environ:
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Key loaded successfully from environment variable.")
else:
    print("❌ Environment variable 'GOOGLE_API_KEY' not found.")
    print("🔑 Please set it in your terminal before running the notebook and restart the kernel.")


❌ Environment variable 'GOOGLE_API_KEY' not found.
🔑 Please set it in your terminal before running the notebook and restart the kernel.


✅ Loaded API Key from .zshrc file.


In [1]:
import os
from google import genai


# 读取.zshrc中的API密钥（仅限macOS/Linux用户）
def load_api_key_from_zshrc():
    zshrc_path = os.path.expanduser("~/.zshrc")
    try:
        with open(zshrc_path, 'r') as file:
            for line in file:
                if line.startswith('export GOOGLE_API_KEY='):
                    key = line.split('=')[1].strip().strip('"').strip("'")
                    return key
    except FileNotFoundError:
        print("❌ .zshrc file not found.")
    return None
if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = load_api_key_from_zshrc()
    print("✅ Loaded API Key from .zshrc file." )

# -*- coding: utf-8 -*-
import json
import time
import logging
import math
import os
import threading
import concurrent.futures
import random
from typing import List

import httpx
from tqdm.notebook import tqdm
from pydantic import BaseModel, Field

# Google Gemini Python SDK（google-genai）
from google import genai
from google.genai import types as genai_types
from google.genai import errors as genai_errors
from google.api_core import exceptions as google_exceptions  # 兼容性保留（如你需沿用）

# ============== 基本配置 ==============
# 代理（如无需可注释）
os.environ.setdefault('HTTP_PROXY', 'http://127.0.0.1:7890')
os.environ.setdefault('HTTPS_PROXY', 'http://127.0.0.1:7890')
print("✅ Proxy configured to use", os.environ.get('HTTP_PROXY', 'N/A'))

NUM_THREADS = 1
INPUT_FILE = "webpath_image_data_robust.json"
OUTPUT_FILE = "enhancements.json"
MODEL_NAME = "gemini-2.5-flash-lite"  # 支持思考模式的快速模型
TOTAL_CHUNKS = 1
API_CALL_RETRIES = 10

# ============== 日志配置 ==============
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(threadName)s - %(levelname)s - %(message)s'
)

# ============== 线程上下文 ==============
enhancements_lock = threading.Lock()
thread_local_data = threading.local()

# ============== Pydantic 模型（结构化输出） ==============
class ClassificationItem(BaseModel):
    id: str = Field(description="The unique identifier (image_url) from the input.")
    tags: List[str] = Field(description="A flat list of 3-7 relevant keywords.")
    classificationPath: List[str] = Field(description="A 4-level hierarchical path: ['System', 'Organ', 'Process', 'Finding'].")

class BatchClassificationResponse(BaseModel):
    classifications: List[ClassificationItem]

# ============== 系统指令（System Instruction） ==============
SYSTEM_PROMPT = """
You are an expert medical pathologist and a meticulous data scientist specializing in systematizing information. You will be given a JSON array of objects, where each object has a unique `id` (which is an image_url) and a `description`. Your task is to process EVERY object in the input array and return a single JSON object containing a list of classifications.

The output JSON object MUST strictly follow this schema:
{
  "classifications": [
    {
      "id": "The original id from the input object",
      "tags": ["Tag1", "Tag2", ...],
      "classificationPath": ["System", "Organ/Region", "Process", "Finding"]
    },
    ...
  ]
}

For each item, follow these classification guidelines with extreme precision:

### 1. `classificationPath` Naming Rules ###

You MUST create a strict 4-level hierarchy. Adhere to the controlled vocabulary provided below for Levels 1 and 3.

**Level 1: Organ System**
You MUST use one of the following exact strings:
- "Cardiovascular System"
- "Respiratory System"
- "Gastrointestinal System"
- "Hepatobiliary System & Pancreas"
- "Renal & Urinary System"
- "Male Genital System"
- "Female Genital System"
- "Breast"
- "Endocrine System"
- "Hematopoietic & Lymphoid System"
- "Central Nervous System"
- "Peripheral Nervous System"
- "Musculoskeletal System" (for bone, joint, soft tissue, muscle)
- "Integumentary System" (for skin)
- "Special Senses" (for eye, ear)
- "General Pathology" (for concepts not tied to one system, e.g., inflammation principles)
- "Systemic & Infectious Disease" (when the primary focus is a systemic process or organism)

**Level 2: Organ or Region**
Use the specific organ or anatomical region mentioned in the text.
*Examples based on Level 1:*
- If "Cardiovascular System", examples: "Heart", "Aorta", "Coronary Artery".
- If "Respiratory System", examples: "Lung", "Pleura", "Larynx".
- If "Gastrointestinal System", examples: "Esophagus", "Stomach", "Colon", "Appendix".
- If "Central Nervous System", examples: "Cerebrum", "Cerebellum", "Spinal Cord", "Meninges".

**Level 3: Pathological Process**
You MUST use one of the following exact strings:
- "Neoplasia - Benign" (for benign tumors like adenomas, leiomyomas)
- "Neoplasia - Malignant" (for cancers like carcinoma, sarcoma, lymphoma)
- "Neoplasia - Precursor/Dysplasia" (for pre-cancerous changes like PIN, CIN, dysplasia)
- "Inflammation - Acute"
- "Inflammation - Chronic"
- "Inflammation - Granulomatous"
- "Infection" (when an organism is the primary finding)
- "Vascular Pathology" (for thrombosis, embolism, atherosclerosis, infarction, hemorrhage)
- "Cellular Injury, Necrosis & Adaptation" (for atrophy, hypertrophy, apoptosis, specific necrosis types)
- "Degenerative & Deposition Disease" (for Alzheimer's, amyloidosis, steatosis)
- "Congenital & Genetic Anomaly" (for birth defects, inherited disorders)
- "Immunopathology" (for autoimmune diseases, hypersensitivity, transplant rejection)
- "Trauma & Injury" (for physical injuries, fractures)
- "Metabolic & Nutritional Disorder"
- "Normal Histology & Anatomy" (use this if the description is of normal tissue)

**Level 4: Specific Finding or Disease**
Use the most specific diagnosis, entity, or morphological finding described in the text.
*Examples:* "Myocardial Infarction", "Infiltrating Ductal Carcinoma", "Tuberculous Granuloma", "Adenomatous Polyp", "Neuritic Plaque", "Aschoff Body", "Normal Seminiferous Tubules".

### 2. `tags` Generation Rules ###

Create a flat list of 3-7 relevant keywords. These tags should include:
- The terms used in all four levels of the `classificationPath`.
- Key morphologic findings mentioned (e.g., "Caseous Necrosis", "Psammoma Bodies", "Hypertrophy", "Crypt Abscess", "Contraction Bands").
- The type of specimen or image if implied (e.g., "Microscopy", "Gross Pathology", "Radiograph", "Autopsy").
- The causative organism if mentioned (e.g., "Mycobacterium tuberculosis", "Candida albicans").

Process all items in the input JSON array and return the complete, structured JSON object.
"""

# ============== 客户端初始化（修复：不再使用 request_options） ==============
def get_gemini_client():
    """
    为每个线程创建并缓存一个 API 客户端实例。
    - 超时/代理等通过 http_options 统一设置（新版 google-genai SDK 推荐方式）。
    - 密钥支持 GEMINI_API_KEY 或 GOOGLE_API_KEY（任一皆可）。
    """
    if not hasattr(thread_local_data, "client"):
        api_key = os.environ.get("GEMINI_API_KEY") or os.environ.get("GOOGLE_API_KEY")
        if not api_key:
            raise ValueError("❌ 未检测到 GEMINI_API_KEY 或 GOOGLE_API_KEY 环境变量。")

        http_options = genai_types.HttpOptions(
            client_args={
                # 全局超时（秒），覆盖每次调用层面的需要
                "timeout": 300.0,
                # 如需自定义代理，也可使用 "proxies": {...} 或 "proxy": "socks5://..."
                # httpx 将自动读取 HTTP_PROXY/HTTPS_PROXY 环境变量
            }
        )

        thread_local_data.client = genai.Client(
            api_key=api_key,
            http_options=http_options,
            # 如需固定 API 版本可指定：api_version="v1"
        )
        logging.info("New Gemini Client created for this thread.")
    return thread_local_data.client

# ============== 解析落盘（去重） ==============
def persist_results(all_enhancements, output_path):
    """以 id 去重后落盘"""
    uniq = {}
    for it in all_enhancements:
        if isinstance(it, dict) and 'id' in it:
            uniq[it['id']] = it
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(list(uniq.values()), f, ensure_ascii=False, indent=2)

# ============== 单批处理（修复 generate_content 调用+结构化兜底） ==============
def process_chunk(chunk: List[dict], chunk_index: int):
    """
    将整个分块作为单个请求发送给 Gemini API（支持思考模式）。
    - 使用 system_instruction 承载规则
    - 结构化输出：优先 .parsed；失败回退 JSON 文本解析
    - 改善异常捕获：errors.APIError 与 httpx 超时
    """
    if not chunk:
        return []

    client = get_gemini_client()
    # 用户内容只放数据，系统规则在 system_instruction
    input_json_string = json.dumps(chunk, ensure_ascii=False, indent=2)
    user_prompt = f"Process the following batch:\n---\n{input_json_string}\n---"

    # 指数退避 + 轻微抖动
    for attempt in range(API_CALL_RETRIES):
        try:
            logging.info(f"Sending chunk {chunk_index} ({len(chunk)} items) to API, attempt {attempt + 1}/{API_CALL_RETRIES}...")

            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=user_prompt,
                config=genai_types.GenerateContentConfig(
                    system_instruction=SYSTEM_PROMPT.strip(),
                    response_mime_type="application/json",
                    response_schema=BatchClassificationResponse,
                    temperature=0,
                    thinking_config=genai_types.ThinkingConfig(thinking_budget=-1),
                    # 如遇偶发截断，可按需加入 max_output_tokens
                    # max_output_tokens=8192,
                ),
            )

            # 结构化优先
            parsed_items = None
            if getattr(response, "parsed", None):
                parsed_items = [item.model_dump() for item in response.parsed.classifications]
            else:
                # 兜底：从原始文本解析
                try:
                    raw = json.loads(response.text)
                    parsed_items = raw.get("classifications", [])
                    if not isinstance(parsed_items, list):
                        raise ValueError("`classifications` is not a list")
                except Exception as e:
                    raise ValueError(f"Structured output empty and response.text not valid JSON. raw={response.text[:200]}...") from e

            # 基本一致性校验
            response_ids = {item.get('id') for item in parsed_items if isinstance(item, dict)}
            input_ids = {item['id'] for item in chunk}
            if response_ids != input_ids:
                missing = input_ids - response_ids
                extra = response_ids - input_ids
                logging.warning(
                    f"ID mismatch in chunk {chunk_index}! "
                    f"Missing: {len(missing)} / Extra: {len(extra)}"
                )

            logging.info(f"Successfully received and validated response for chunk {chunk_index}.")
            return parsed_items

        except genai_errors.APIError as e:
            # API 层错误（4xx/5xx）
            logging.warning(
                f"API error on chunk {chunk_index} (Attempt {attempt+1}): "
                f"code={getattr(e, 'code', '?')} msg={getattr(e, 'message', str(e))}"
            )
            # 遇到明显的非暂态错误（如 400 参数问题），不必反复重试
            code = getattr(e, 'code', None)
            if code in (400, 401, 403, 404):
                break
            # 暂态错误退避
            sleep_s = (3 ** attempt) + random.uniform(0, 0.5)
            time.sleep(sleep_s)

        except (httpx.ReadTimeout, httpx.ConnectTimeout, concurrent.futures.TimeoutError) as e:
            logging.warning(
                f"Network timeout on chunk {chunk_index} (Attempt {attempt+1}). "
                f"Error: {e}. Retrying..."
            )
            sleep_s = (3 ** attempt) + random.uniform(0, 0.5)
            time.sleep(sleep_s)

        except Exception as e:
            logging.warning(
                f"API call for chunk {chunk_index} failed (Attempt {attempt+1}). "
                f"Error: {e}. Retrying..."
            )
            sleep_s = (3 ** attempt) + random.uniform(0, 0.5)
            time.sleep(sleep_s)

    logging.error(f"Failed to process chunk {chunk_index} after {API_CALL_RETRIES} attempts.")
    return None

# ============== 主执行逻辑（保持大批量并发、分块由 TOTAL_CHUNKS 控制） ==============
def generate_enhancements():
    # 支持两种环境变量名
    if not (os.environ.get('GEMINI_API_KEY') or os.environ.get('GOOGLE_API_KEY')):
        logging.error("❌ API Key not found. Please set GEMINI_API_KEY or GOOGLE_API_KEY.")
        return

    # 读入数据
    try:
        with open(INPUT_FILE, 'r', encoding='utf-8') as f:
            all_data = json.load(f)
        logging.info(f"Loaded {len(all_data)} pages from '{INPUT_FILE}'.")
    except (FileNotFoundError, json.JSONDecodeError) as e:
        logging.error(f"Could not read or parse file: {e}")
        return

    # 以 image_url 作为唯一 ID，拼接 descriptions
    items_to_process = [
        {"id": img['image_url'], "description": " ".join(img['descriptions'])}
        for page in all_data if page.get('extracted_data')
        for img in page['extracted_data'] if img.get('descriptions') and img.get('image_url')
    ]

    # 断点续跑：加载已有结果
    all_enhancements = []
    processed_ids = set()
    if os.path.exists(OUTPUT_FILE):
        try:
            with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
                all_enhancements = json.load(f)
                processed_ids = {item['id'] for item in all_enhancements if isinstance(item, dict) and 'id' in item}
            logging.info(f"Resuming. Loaded {len(all_enhancements)} existing enhancements.")
        except Exception as e:
            logging.warning(f"Could not load existing enhancements. Starting fresh. Error: {e}")

    # 过滤已处理
    items_to_process = [item for item in items_to_process if item['id'] not in processed_ids]
    logging.info(f"Remaining items to process: {len(items_to_process)}")
    if not items_to_process:
        logging.info("All items already processed.")
        return

    # 大批量策略：按 TOTAL_CHUNKS 切分，单批可能较大（符合你的说明）
    chunk_size = math.ceil(len(items_to_process) / TOTAL_CHUNKS)
    chunk_size = max(1, chunk_size)
    chunks_to_run = [items_to_process[i:i + chunk_size] for i in range(0, len(items_to_process), chunk_size)]

    with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_THREADS, thread_name_prefix='GeminiWorker') as executor:
        future_to_chunk_index = {executor.submit(process_chunk, chunk, i+1): i+1 for i, chunk in enumerate(chunks_to_run)}

        with tqdm(total=len(chunks_to_run), desc="Processing Chunks") as pbar:
            for future in concurrent.futures.as_completed(future_to_chunk_index):
                chunk_index = future_to_chunk_index[future]
                pbar.set_description(f"Chunks | Finished: {pbar.n+1}/{pbar.total} | Current: Chunk {chunk_index}")
                try:
                    results_list = future.result()
                    if results_list is not None:
                        with enhancements_lock:
                            all_enhancements.extend(results_list)
                            persist_results(all_enhancements, OUTPUT_FILE)
                        logging.info(f"Chunk {chunk_index} completed and saved. Total items now: {len(all_enhancements)}")
                    else:
                        logging.error(f"Chunk {chunk_index} returned no results.")
                except Exception as exc:
                    logging.error(f"Chunk {chunk_index} generated an exception: {exc}")
                pbar.update(1)

    logging.info(f"✅ All chunks processed. Final enhancements saved to '{OUTPUT_FILE}'.")

# ============== 运行入口 ==============
if __name__ == "__main__":
    generate_enhancements()


2025-10-26 12:43:05,733 - MainThread - INFO - Loaded 2389 pages from 'webpath_image_data_robust.json'.
2025-10-26 12:43:05,738 - MainThread - INFO - Resuming. Loaded 1796 existing enhancements.
2025-10-26 12:43:05,739 - MainThread - INFO - Remaining items to process: 1


✅ Loaded API Key from .zshrc file.
✅ Proxy configured to use http://127.0.0.1:7890


Processing Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-26 12:43:05,918 - GeminiWorker_0 - INFO - New Gemini Client created for this thread.
2025-10-26 12:43:05,920 - GeminiWorker_0 - INFO - Sending chunk 1 (1 items) to API, attempt 1/10...
2025-10-26 12:43:05,922 - GeminiWorker_0 - INFO - AFC is enabled with max remote calls: 10.
2025-10-26 12:43:09,192 - GeminiWorker_0 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-10-26 12:43:09,198 - GeminiWorker_0 - INFO - Successfully received and validated response for chunk 1.
2025-10-26 12:43:09,230 - MainThread - INFO - Chunk 1 completed and saved. Total items now: 1797
2025-10-26 12:43:09,233 - MainThread - INFO - ✅ All chunks processed. Final enhancements saved to 'enhancements.json'.


## 验证

In [2]:
# verify_missing_ids.py
import json

INPUT_FILE = "webpath_image_data_robust.json"
OUTPUT_FILE = "enhancements.json"

def load_input_ids():
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        pages = json.load(f)
    ids = [
        img["image_url"]
        for page in pages if page.get("extracted_data")
        for img in page["extracted_data"] if img.get("descriptions") and img.get("image_url")
    ]
    return set(ids)

def load_output_ids():
    with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
        out = json.load(f)
    return {x["id"] for x in out if isinstance(x, dict) and "id" in x}

if __name__ == "__main__":
    in_ids = load_input_ids()
    out_ids = load_output_ids()
    missing = sorted(in_ids - out_ids)
    extra = sorted(out_ids - in_ids)

    print(f"Total input ids: {len(in_ids)}")
    print(f"Total output ids: {len(out_ids)}")
    print(f"Missing count: {len(missing)}")
    print(f"Extra count: {len(extra)}")

    if missing:
        print("\nMissing ids (up to 50):")
        for mid in missing[:50]:
            print(mid)


Total input ids: 1797
Total output ids: 1797
Missing count: 0
Extra count: 0


In [3]:
import json

INPUT_FILE = "webpath_image_data_robust.json"

def count_unique_image_urls(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    urls = [
        img["image_url"]
        for page in data if page.get("extracted_data")
        for img in page["extracted_data"] if img.get("image_url")
    ]
    
    unique_urls = set(urls)
    
    print(f"Total image_url entries (including duplicates): {len(urls)}")
    print(f"Unique image_url count: {len(unique_urls)}")
    print(f"Duplicate count: {len(urls) - len(unique_urls)}")

if __name__ == "__main__":
    count_unique_image_urls(INPUT_FILE)


Total image_url entries (including duplicates): 2442
Unique image_url count: 1945
Duplicate count: 497


In [4]:
import json

INPUT_FILE = "webpath_image_data_robust.json"

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

urls_with_desc = set()
urls_without_desc = set()

for page in data:
    if not page.get("extracted_data"):
        continue
    for img in page["extracted_data"]:
        url = img.get("image_url")
        if not url:
            continue
        if img.get("descriptions"):
            urls_with_desc.add(url)
        else:
            urls_without_desc.add(url)

print(f"Total unique URLs: {len(urls_with_desc | urls_without_desc)}")
print(f"URLs with descriptions: {len(urls_with_desc)}")
print(f"URLs without descriptions: {len(urls_without_desc)}")
print(f"Coverage of described images: {len(urls_with_desc) / len(urls_with_desc | urls_without_desc):.2%}")


Total unique URLs: 1945
URLs with descriptions: 1797
URLs without descriptions: 192
Coverage of described images: 92.39%


## 合并

In [4]:
import json
import logging

# --- 配置 ---
ORIGINAL_DATA_FILE = "webpath_image_data_robust.json"
ENHANCEMENTS_FILE = "enhancements.json"
FINAL_OUTPUT_FILE = "webpath_final_enriched_data.json"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def merge_data():
    logging.info("--- Starting Final Merge Process ---")
    
    try:
        with open(ORIGINAL_DATA_FILE, 'r', encoding='utf-8') as f: original_data = json.load(f)
        with open(ENHANCEMENTS_FILE, 'r', encoding='utf-8') as f: enhancements = json.load(f)
        logging.info(f"Loaded {len(original_data)} pages and {len(enhancements)} enhancements.")
    except FileNotFoundError as e:
        logging.error(f"Could not find input file: {e}."); return

    # --- 关键修改：使用 'id' (现在是image_url) 作为键 ---
    enhancements_map = {item['id']: item for item in enhancements}
    logging.info("Created enhancement map for efficient lookup.")

    final_data = []
    
    for page in original_data:
        if not page.get('extracted_data'): continue
            
        new_page_obj = {
            "id": page['url'].split('/')[-1],
            "url": page['url'],
            "extracted_data": []
        }
        
        for image_item in page['extracted_data']:
            # --- 关键修改：使用 image_url 进行匹配 ---
            enhancement = enhancements_map.get(image_item['image_url'])
            
            clean_image_item = {
                "image_url": image_item["image_url"],
                "descriptions": image_item["descriptions"]
            }
            
            if enhancement:
                clean_image_item["tags"] = enhancement.get("tags", [])
                clean_image_item["classificationPath"] = enhancement.get("classificationPath", [])
            
            new_page_obj['extracted_data'].append(clean_image_item)
        
        if new_page_obj['extracted_data']:
            final_data.append(new_page_obj)

    logging.info(f"Merged {len(enhancements_map)} enhancements into final data structure.")

    with open(FINAL_OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_data, f, indent=2)
        
    print("-" * 50)
    print(f"✅ Merge complete! Final enriched data saved to '{FINAL_OUTPUT_FILE}'")
    print(f"Total pages in final file: {len(final_data)}")
    print("-" * 50)

# --- 运行合并函数 ---
merge_data()

2025-10-26 12:43:46,696 - MainThread - INFO - --- Starting Final Merge Process ---
2025-10-26 12:43:46,722 - MainThread - INFO - Loaded 2389 pages and 1797 enhancements.
2025-10-26 12:43:46,723 - MainThread - INFO - Created enhancement map for efficient lookup.
2025-10-26 12:43:46,728 - MainThread - INFO - Merged 1797 enhancements into final data structure.


--------------------------------------------------
✅ Merge complete! Final enriched data saved to 'webpath_final_enriched_data.json'
Total pages in final file: 2388
--------------------------------------------------


# API连接性测试

In [10]:
pip install -U google-genai

Note: you may need to restart the kernel to use updated packages.


In [11]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite", contents="Explain how AI works in a few words"
)
print(response.text)

2025-10-26 03:24:33,083 - INFO - AFC is enabled with max remote calls: 10.
2025-10-26 03:24:35,145 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


AI learns from data to make predictions or decisions.


In [6]:
# 对于 macOS/Linux
!curl -x http://127.0.0.1:7890 https://www.google.com



<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="zh-TW"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="Hca64rU7zXLxNifVPBPndw">(function(){var _g={kEI:'lyP9aOvUFe7t1e8P7-25YA',kEXPI:'0,18167,184687,2,96,5,3800553,236535,48791,30022,16105,201864,142932,42344368,25228681,78017,74377,8928,13974,42251,30634,9139,4600,328,6225,37821,26344,15049,8221,7413,30376,28339,54206,353,18880,13584,5773,27612,4719,11805,6251,35,3420,13483,12108,5683,3604,591,17181,3346,15934,15321,551,4803,1218,1,1552,7355,727,1033,6090,10089,2747,2,4,1,320,844,4256,670,2260,10027,730,355,332,3348,2477,2,8109,4215,1490,406,185,4,1378,136,3355,195,4261,1554,2,1347,5,682,3965,17,236,1169,985,4,5,2633,4874,2507,12,4,1261,600,2,876,2229,4496,593,19,533,6,2469,16,779,3133,3,936,1531,1475,2703,242,9531,3700,8722,294,3322,210,25,796,243,2,4,1,1